In [1]:
import os
import zipfile
import cv2
from ultralytics import YOLO

In [2]:
# --- Configuration for Local Environment ---
PROJECT_ROOT = os.getcwd() # Assumes you run this from the project folder
ZIP_FILE_NAME = r'YOLO_URBAN_ISSUES_DATASET.zip'
DATASET_PATH = os.path.join(PROJECT_ROOT, r'YOLO_URBAN_ISSUES_DATASET')
ZIP_FILE_PATH = os.path.join(PROJECT_ROOT, ZIP_FILE_NAME)

In [3]:
# Check if the dataset is already unzipped
if not os.path.exists(DATASET_PATH):
    print(f"Dataset not found at {DATASET_PATH}. Starting unzipping process...")
    
    if os.path.exists(ZIP_FILE_PATH):
        try:
            with zipfile.ZipFile(ZIP_FILE_PATH, 'r') as zip_ref:
                # Extract all contents to the PROJECT_ROOT
                zip_ref.extractall(PROJECT_ROOT)
            print(f"Successfully extracted {ZIP_FILE_NAME} to {PROJECT_ROOT}.")
            
            # --- IMPORTANT: Standardize Folder Name ---
            # Check for a common issue: the zip creates a directory named after the file.
            possible_unzip_name = ZIP_FILE_NAME.replace('.zip', '')
            possible_unzip_path = os.path.join(PROJECT_ROOT, possible_unzip_name)

            if os.path.exists(possible_unzip_path) and not os.path.exists(DATASET_PATH):
                os.rename(possible_unzip_path, DATASET_PATH)
                print(f"Renamed {possible_unzip_name} to {os.path.basename(DATASET_PATH)}")
                
        except Exception as e:
            print(f"Error during unzipping: {e}")
            
    else:
        print(f"Error: ZIP file not found at {ZIP_FILE_PATH}. Please ensure '{ZIP_FILE_NAME}' is in the project root.")

else:
    print(f"Dataset already found and ready at: {DATASET_PATH}")

Dataset already found and ready at: d:\Projects\Sahaay\YOLO_URBAN_ISSUES_DATASET


In [4]:
import os

# --- Configuration ---
# Your verified absolute path (which seems correct now)
YOUR_VERIFIED_ABSOLUTE_PATH = r' D:/Projects/SIH/YOLO_URBAN_ISSUES_DATASET/YOLO_URBAN_ISSUES_DATASET' 
# ---------------------

# Define the content for the YOLO configuration file
yaml_content = f"""
# YOLOv8 custom data configuration
path: {YOUR_VERIFIED_ABSOLUTE_PATH}
train: images/train
val: images/val # <-- ENSURE THIS IS 'val'

# Number of classes
nc: 9

# Class names based on your description
names:
  0: Damaged concrete structures
  1: DamagedElectricalPoles
  2: DamagedRoadSigns
  3: DeadAnimalsPollution
  4: FallenTrees
  5: Garbage
  6: Graffitti
  7: IllegalParking
  8: Potholes and RoadCracks
"""

yaml_file_path = 'data.yaml'
with open(yaml_file_path, 'w') as f:
    f.write(yaml_content)

print(f"Configuration file '{yaml_file_path}' confirmed with val path: images/val")

Configuration file 'data.yaml' confirmed with val path: images/val


In [5]:
import os

# --- 1. Define the suspected root directory based on your error message ---
# This is the full path where your unzipped dataset folder should be located.
# We will use the path shown in your error message, but you might need to adjust it
# if the unzipping created a different folder name (e.g., 'urban_issues_dataset').

# Check for the two most common unzipped folder names:
possible_paths = [
    r'D:/Projects/SIH/YOLO_URBAN_ISSUES_DATASET/YOLO_URBAN_ISSUES_DATASET', # Path seen in the error message
    r'D:/Projects/SIH/urban_issues_dataset',     # Path suggested in previous steps
]

found_path = None
for p in possible_paths:
    if os.path.isdir(p):
        found_path = p
        break

if found_path:
    print(f"FOUND DATASET AT: {found_path}")
    print(f"   Please use this path for the 'path:' variable in your YAML file.")
else:
    print("Dataset folder not found at the suspected locations.")
    print("   Please manually check where the 'urban_datasets.zip' file extracted.")
    # Fallback to current working directory assumption
    print(f"If it extracted to your current directory, try: {os.path.join(os.getcwd(), 'urban_issues_dataset')}")

Dataset folder not found at the suspected locations.
   Please manually check where the 'urban_datasets.zip' file extracted.
If it extracted to your current directory, try: d:\Projects\Sahaay\urban_issues_dataset


In [6]:
import os

# UPDATE THIS LINE with the absolute path you found in Step 1 
# Example: DATASET_ROOT = r'D:\Projects\SIH\urban_datasets' 
DATASET_ROOT = r'D:\Projects\SIH\YOLO_URBAN_ISSUES_DATASET\YOLO_URBAN_ISSUES_DATASET' # Starting with the current path as a guess

# Initialize lists to avoid NameError if directories don't exist
image_files = []
label_files = []

# Directories to check
TRAIN_IMAGES_DIR = os.path.join(DATASET_ROOT, 'images', 'train')
TRAIN_LABELS_DIR = os.path.join(DATASET_ROOT, 'labels', 'train')

# 1. Check Image folder contents
print("--- Checking Training Images Directory ---")
if not os.path.exists(TRAIN_IMAGES_DIR):
    print(f"ERROR: Training Images directory does not exist at: {TRAIN_IMAGES_DIR}")
else:
    # Check if the path points to files or a directory containing files
    image_files = [f for f in os.listdir(TRAIN_IMAGES_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
    print(f"Found {len(image_files)} image files in: {TRAIN_IMAGES_DIR}")
    if len(image_files) > 0:
        print(f"   First 3 files: {image_files[:3]}")

# 2. Check Labels folder contents
print("\n--- Checking Training Labels Directory ---")
if not os.path.exists(TRAIN_LABELS_DIR):
    print(f"ERROR: Training Labels directory does not exist at: {TRAIN_LABELS_DIR}")
else:
    label_files = [f for f in os.listdir(TRAIN_LABELS_DIR) if f.lower().endswith('.txt')]
    print(f"Found {len(label_files)} label files in: {TRAIN_LABELS_DIR}")
    if len(label_files) > 0:
        print(f"   First 3 files: {label_files[:3]}")

# 3. Final Check: Image/Label Count Match
if len(image_files) > 0 and len(label_files) > 0 and len(image_files) != len(label_files):
    print(f"\nWARNING: Image count ({len(image_files)}) does not match label count ({len(label_files)}).")
    print("This will cause training errors later, but not the current FileNotFoundError.")
elif len(image_files) == 0 and os.path.exists(TRAIN_IMAGES_DIR):
    print("\nCRITICAL: The images/train folder exists but is EMPTY or contains only unsupported files.")
elif len(label_files) == 0 and os.path.exists(TRAIN_LABELS_DIR):
    print("\nCRITICAL: The labels/train folder exists but is EMPTY or missing .txt files.")

print("\n--- Diagnostic Complete ---")

--- Checking Training Images Directory ---
ERROR: Training Images directory does not exist at: D:\Projects\SIH\YOLO_URBAN_ISSUES_DATASET\YOLO_URBAN_ISSUES_DATASET\images\train

--- Checking Training Labels Directory ---
ERROR: Training Labels directory does not exist at: D:\Projects\SIH\YOLO_URBAN_ISSUES_DATASET\YOLO_URBAN_ISSUES_DATASET\labels\train

--- Diagnostic Complete ---


In [7]:
import os

for root, dirs, files in os.walk("YOLO_URBAN_ISSUES_DATASET"):
    if "images" in dirs and "labels" in dirs:
        print("FOUND DATASET ROOT:", root)


FOUND DATASET ROOT: YOLO_URBAN_ISSUES_DATASET\YOLO_URBAN_ISSUES_DATASET


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.train(
    data="data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    device="cpu"  
)


d:\Projects\Sahaay\.venv\Lib\site-packages\ultralytics\nn\tasks.py:732: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")


New https://pypi.org/project/ultralytics/8.4.6 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.0  Python-3.12.6 torch-2.5.1+cu121 CPU (AMD Ryzen 5 7535HS with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train23, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frame

d:\Projects\Sahaay\.venv\Lib\site-packages\ultralytics\engine\trainer.py:261: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning D:\Projects\Sahaay\YOLO_URBAN_ISSUES_DATASET\YOLO_URBAN_ISSUES_DATASET\labels\train... 377 images, 49 backgrounds, 0 corrupt: 100%|██████████| 377/377 [00:01<00:00, 254.57it/s]

train: WARNING  D:\Projects\Sahaay\YOLO_URBAN_ISSUES_DATASET\YOLO_URBAN_ISSUES_DATASET\images\train\30717230707_cc09c3e457_b_jpg.rf.1d5a1334fabe41fd6f5f19a2d25ac002.jpg: 1 duplicate labels removed
train: WARNING  D:\Projects\Sahaay\YOLO_URBAN_ISSUES_DATASET\YOLO_URBAN_ISSUES_DATASET\images\train\30717230707_cc09c3e457_b_jpg.rf.65c3ceebe756b8b3ab49c0987e78d404.jpg: 2 duplicate labels removed
train: WARNING  D:\Projects\Sahaay\YOLO_URBAN_ISSUES_DATASET\YOLO_URBAN_ISSUES_DATASET\images\train\392C51FF00000578-0-image-a-11_1475781587245_jpg.rf.40a8bda830db6d5dba20687d03b78dc6.jpg: 1 duplicate labels removed
train: WARNING  D:\Projects\Sahaay\YOLO_URBAN_ISSUES_DATASET\YOLO_URBAN_ISSUES_DATASET\images\train\392C51FF00000578-0-image-a-11_1475781587245_jpg.rf.c30fe8120eeb1110fbd36256c83ad6ad.jpg: 2 duplicate labels removed
train: WARNING  D:\Projects\Sahaay\YOLO_URBAN_ISSUES_DATASET\YOLO_URBAN_ISSUES_DATASET\images\train\AdobeStock_399405401_jpeg.rf.02a410708ac253f90bc032d267d01c27.jpg: 4 dupli

train: New cache created: D:\Projects\Sahaay\YOLO_URBAN_ISSUES_DATASET\YOLO_URBAN_ISSUES_DATASET\labels\train.cache


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      18/50         0G      1.331      1.832      1.516         39        640: 100%|██████████| 24/24 [02:08<00:00,  5.36s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:07<00:00,  2.65s/it]

                   all         66         72      0.704      0.387      0.397      0.257



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
# mkdir sr_models

## The -L flag handles redirects, and -o specifies the output filename/path
# curl -L -o sr_models/EDSR_x4.pb https://github.com/opencv/opencv_extra/raw/master/testdata/dnn_superres/EDSR_x4.pb

# echo "Downloaded EDSR_x4.pb to sr_models/ folder."